In [27]:
from fairseq.models.bart import BARTModel

In [28]:
# # MBart Test
# mbart = BARTModel.from_pretrained('mbart.cc25.v2', checkpoint_file='model.pt')
# mbart.eval()

In [29]:
# # This is needed for the other four languages
# from fairseq.tasks import register_task
# from fairseq.tasks.translation import TranslationTask


# @register_task('translation_without_lang_token')
# class TranslationPLBARTTask(TranslationTask):
#     @staticmethod
#     def add_args(parser):
#         """Add task-specific arguments to the parser."""
#         TranslationTask.add_args(parser)
#         parser.add_argument('--langs', required=True, metavar='LANG',
#                             help='comma-separated list of monolingual language, '
#                                  'for example, "en,de,fr". These should match the '
#                                  'langs from pretraining (and be in the same order). '
#                                  'You should always add all pretraining language idx '
#                                  'during finetuning.')

#     def __init__(self, args, src_dict, tgt_dict):
#         super().__init__(args, src_dict, tgt_dict)
#         self.langs = args.langs.split(",")
#         for d in [self.src_dict, self.tgt_dict]:
#             for l in self.langs:
#                 d.add_symbol("[{}]".format(l))
#             d.add_symbol("<mask>")


In [30]:
plbart = BARTModel.from_pretrained('PLBART/plbart-java-en_XX', checkpoint_file='model.pt')
plbart.eval()

BARTHubInterface(
  (model): BARTModel(
    (encoder): TransformerEncoder(
      (dropout_module): FairseqDropout()
      (embed_tokens): Embedding(50005, 768, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 768, padding_idx=1)
      (layernorm_embedding): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (dropout_module): FairseqDropout()
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout_module): FairseqDropout()
          (activation_dropout_module): FairseqDropout()
     

In [31]:
len(plbart.task.source_dictionary)

50005

In [32]:
for i in [0,1,2,3,50001,50002, 50003, 50004]:
    print(i, plbart.task.source_dictionary[i])

0 <s>
1 <pad>
2 </s>
3 <unk>
50001 [java]
50002 [python]
50003 [en_XX]
50004 <mask>


In [33]:
fs_model = plbart.model

In [34]:
plbart.args

Namespace(activation_dropout=0.0, activation_fn='gelu', adam_betas='(0.9, 0.98)', adam_eps=1e-06, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='mbart_base', attention_dropout=0.1, best_checkpoint_metric='bleu', bf16=False, bpe='gpt2', broadcast_buffers=False, bucket_cap_mb=25, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='/home/crocoder/Desktop/transformers/PLBART/plbart-java-en_XX', data_buffer_size=10, dataset_impl=None, ddp_backend='no_c10d', decoder_attention_heads=12, decoder_embed_dim=768, decoder_embed_path=None, decoder_ffn_embed_dim=3072, decoder_input_dim=768, decoder_layerdrop=0, decoder_layers=6, decoder_layers_to_keep=None, decoder_learned_pos=True, decoder_normalize_before=False, decoder_output_dim=768, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_s

In [35]:
from transformers import PLBartConfig, PLBartForConditionalGeneration

In [36]:
hf_model = PLBartForConditionalGeneration.from_pretrained('plbart-java-en_XX')

## Inputs

In [37]:
import sentencepiece as spm

In [38]:
vocab_filepath = "./PLBART/plbart_orig_pretrained_ckpt/sentencepiece.bpe.model"
tokenizer = spm.SentencePieceProcessor()
tokenizer.Load(vocab_filepath)
tokenizer.SetEncodeExtraOptions("")

True

In [39]:
texts = ["This is a sample text", "Another example here"]
text_0_tokens = tokenizer.EncodeAsIds(texts[0].strip())
text_1_tokens = tokenizer.EncodeAsIds(texts[1].strip())
# Need to check how is the original tokenizer

In [40]:
text_1_tokens += [1]*2

In [41]:
import numpy as np
import torch
import torch.nn as nn

In [42]:
input_ids = torch.from_numpy(np.array([text_0_tokens, text_1_tokens]))
attention_mask = torch.ones_like(input_ids)
attention_mask[1, -2:] = 0
token_type_ids = torch.zeros_like(input_ids)

# Encoder Embeddings

## Fairseq

In [43]:
fs_embeds, embed = fs_model.encoder.forward_embedding(input_ids)

In [44]:
# fs_model.encoder.quant_noise # None
# fs_model.encoder.layernorm_embedding #  not None
# fs_model.encoder.embed_positions # not None

## HuggingFace

In [45]:
hf_model.model.encoder.training

False

In [47]:
input_shape = input_ids.size()
inputs_embeds = hf_model.model.encoder.embed_tokens(input_ids) * hf_model.model.encoder.embed_scale
embed_pos = hf_model.model.encoder.embed_positions(input_shape)
hf_embeds = inputs_embeds + embed_pos
hf_embeds = hf_model.model.encoder.layernorm_embedding(hf_embeds)
hf_embeds = nn.functional.dropout(hf_embeds, p= hf_model.model.encoder.dropout, training= hf_model.model.encoder.training)


In [48]:
torch.allclose(hf_embeds[0], fs_embeds[0], atol=1e-5)

True

In [49]:
torch.allclose(hf_embeds[1,:-2], fs_embeds[1, :-2], atol=1e-5)

True

In [50]:
hf_embeds[1, -2:]

tensor([[-0.1719, -0.2876, -0.5173,  ..., -0.3609, -0.7324, -0.4892],
        [-0.2384, -0.3083, -0.6778,  ...,  0.1107, -0.5506, -0.4960]],
       grad_fn=<SliceBackward>)

In [51]:
fs_embeds[1, -2:] # Fairseq handles padding tokens differently, so they won't match.

tensor([[ 0.2318, -0.5792, -0.5779,  ..., -0.1333, -0.8399,  0.0933],
        [ 0.2318, -0.5792, -0.5779,  ..., -0.1333, -0.8399,  0.0933]],
       grad_fn=<SliceBackward>)

In [52]:
torch.allclose(hf_embeds[1,-2:], fs_embeds[1, -2:], atol=1e-5)

False

# Encoder Model

## Fairseq

In [53]:
fs_encoder = fs_model.encoder

In [54]:
src_lengths = torch.tensor([len(text_0_tokens), len(text_1_tokens)])

In [55]:
fs_encoder_out = fs_encoder(input_ids, src_lengths)

In [56]:
fs_encoder_out_encoder_out = fs_encoder_out.encoder_out.permute(1, 0, 2)

In [57]:
fs_encoder_out_encoder_out.shape

torch.Size([2, 5, 768])

## HuggingFace

In [58]:
hf_encoder = hf_model.model.encoder

In [59]:
hf_encoder_out = hf_encoder(input_ids, attention_mask)

In [60]:
hf_encoder_out = hf_encoder_out.last_hidden_state

In [61]:
torch.allclose(hf_encoder_out[0], fs_encoder_out_encoder_out[0], atol=1e-5)

True

In [62]:
torch.allclose(hf_encoder_out[1,:-2], fs_encoder_out_encoder_out[1,:-2], atol=1e-5)

True

# Decoder

## Fairseq

In [63]:
fs_decoder = fs_model.decoder

In [64]:
fs_decoder_out = fs_decoder(input_ids, fs_encoder_out)

In [65]:
len(fs_decoder_out)
fs_decoder_out_decoder_out = fs_decoder_out[0]

In [66]:
fs_decoder_out[1].keys()

dict_keys(['attn', 'inner_states'])

In [67]:
#  fs_decoder_out[1]['inner_states'][-1].permute(1, 0, 2).shape

In [68]:
fs_decoder_out_inner_state = fs_decoder_out[1]['inner_states'][-1].permute(1, 0, 2)

In [69]:
fs_decoder_out_decoder_out.shape

torch.Size([2, 5, 50005])

## HuggingFace

In [70]:
hf_decoder = hf_model.model.decoder

In [71]:
hf_decoder_out = hf_decoder(input_ids, attention_mask=attention_mask, encoder_hidden_states=hf_encoder_out, encoder_attention_mask=attention_mask)

In [72]:
hf_decoder_out.last_hidden_state.shape

torch.Size([2, 5, 768])

In [73]:
torch.allclose(hf_decoder_out.last_hidden_state[0], fs_decoder_out_inner_state[0], atol=1e-5)

True

In [74]:
torch.allclose(hf_decoder_out.last_hidden_state[1, :-2], fs_decoder_out_inner_state[1, :-2], atol=1e-5)

True

In [75]:
lm_logits = hf_model.lm_head(hf_decoder_out.last_hidden_state) + hf_model.final_logits_bias

In [76]:
lm_logits.shape

torch.Size([2, 5, 50005])

In [77]:
torch.allclose(lm_logits[0], fs_decoder_out_decoder_out[0], atol=1e-5)

True

In [78]:
torch.allclose(lm_logits[1, :-2], fs_decoder_out_decoder_out[1, :-2], atol=1e-5)

True